# AgentCore ランタイムにおけるランタイムコンテキストとセッション管理の理解

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore ランタイムにおけるランタイムコンテキストとセッション管理について理解し、活用する方法を学びます。この例では、AgentCore ランタイムがセッションを処理し、複数の呼び出しにわたってコンテキストを維持する方法、そしてエージェントがコンテキストオブジェクトを通じてランタイム情報にアクセスする方法を示します。

Amazon Bedrock AgentCore ランタイムは、各ユーザーインタラクションに対して分離されたセッションを提供し、エージェントが複数の呼び出しにわたってコンテキストと状態を維持しながら、異なるユーザー間で完全なセキュリティ分離を確保します。

### チュートリアルの詳細

|情報| 詳細|
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ       | コンテキストとセッション管理|
| エージェントタイプ          | シングル         |
| エージェンティックフレームワーク   | Strands Agents |
| LLM モデル           | Anthropic Claude Sonnet 4 |
| チュートリアルコンポーネント | ランタイムコンテキスト、セッション管理、AgentCore ランタイム、Strands Agent および Amazon Bedrock モデル |
| チュートリアル分野   | 分野横断的                                                                   |
| 例の複雑さ  | 中級                                                                     |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK および boto3|

### チュートリアルのアーキテクチャ

このチュートリアルでは、Amazon Bedrock AgentCore ランタイムがセッションを管理し、エージェントにコンテキストを提供する方法を探ります。以下を実演します：

1. **セッションの継続性**：同じセッション ID が複数の呼び出しにわたってコンテキストを維持する方法
2. **コンテキストオブジェクト**：エージェントがコンテキストパラメータを通じてランタイム情報にアクセスする方法
3. **セッション分離**：異なるセッション ID が完全に分離された環境を作成する方法
4. **ペイロードの柔軟性**：ペイロードを通じてエージェントにカスタムデータを渡す方法

デモンストレーションのために、これらのセッション管理機能を示す Strands Agent を使用します。

    
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### チュートリアルの主な機能

* **セッションベースのコンテキスト管理**：AgentCore ランタイムがセッション内でコンテキストを維持する方法の理解
* **ランタイムセッションのライフサイクル**：セッションの作成、維持、終了について学ぶ
* **コンテキストオブジェクトアクセス**：コンテキストパラメータを通じてセッション ID などのランタイム情報にアクセスする
* **セッション分離**：異なるセッションが完全な分離を提供する方法の実証
* **ペイロード処理**：カスタムペイロード構造を通じた柔軟なデータ受け渡し
* **呼び出し間の状態維持**：同じセッション内の複数の呼び出しにわたるエージェント状態の維持

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が稼働していること

## Amazon Bedrock AgentCore ランタイムセッションの理解

コードに入る前に、Amazon Bedrock AgentCore ランタイムがセッションをどのように管理するかを理解することが重要です：

### セッションの分離とセキュリティ

AgentCore ランタイムは専用の microVM を通じて**完全なセッション分離**を提供します：

- **専用リソース**：各セッションは独自の microVM で実行され、CPU、メモリ、ファイルシステムが分離されています
- **セキュリティ境界**：ユーザーセッション間の完全な分離によりデータ汚染を防止します
- **決定論的なクリーンアップ**：セッション完了後、microVM は終了し、メモリはサニタイズされます

### セッションのライフサイクル

AgentCore ランタイムのセッションは特定のライフサイクルに従います：

1. **作成**：最初の呼び出し時に一意の `runtimeSessionId` で新しいセッションが作成されます
2. **アクティブ状態**：セッションはリクエストを処理し、コンテキストを維持します
3. **アイドル状態**：セッションはコンテキストを保持しながら次の呼び出しを待ちます
4. **終了**：以下の理由でセッションが終了します：
   - 非アクティブ状態（15 分間）
   - 最大寿命（8 時間）
   - ヘルスチェックの失敗

### コンテキストの永続性

セッション内で、AgentCore ランタイムは以下を維持します：
- **会話履歴**：以前のやり取りと応答
- **アプリケーション状態**：実行中に作成された変数とオブジェクト
- **ファイルシステム**：セッション中に作成または変更されたファイル
- **環境変数**：カスタム設定と構成

### セッション管理のベストプラクティス

- **一意のセッション ID**：各ユーザーまたは会話に対して一意のセッション ID を生成する
- **コンテキストの再利用**：コンテキストを維持するために関連する呼び出しには同じセッション ID を使用する
- **セッション境界**：異なるユーザーや無関係な会話には異なるセッション ID を使用する
- **一時的な性質**：永続的なデータストレージにセッションを頼らない（永続性には AgentCore Memory を使用する）

In [ ]:
#!uv add -r requirements.txt --active

import os
os.environ["AWS_PROFILE"] = "cline2"

## AgentCore Runtime へのエージェントのデプロイ準備

ここでは、セッション管理とコンテキスト処理を実証するために、エージェントを AgentCore Runtime にデプロイしましょう。私たちのエージェントは以下の方法を示します：

1. **ランタイムコンテキストへのアクセス**： `context` パラメータを使用してセッション情報を取得する
2. **カスタムペイロードの処理**： ペイロードを通じて渡される構造化データを処理する
3. **セッション状態の維持**： セッション内のユーザーインタラクションを追跡する
4. **セッション境界の実証**： 異なるセッションがどのように分離されているかを示す

### Context オブジェクトの理解

AgentCore Runtime の `context` オブジェクトは、現在の実行環境に関する貴重な情報を提供します：

- **session_id**： 現在のランタイムセッション識別子
- **ランタイムメタデータ**： ランタイム環境に関する情報
- **実行詳細**： 現在の呼び出しに関するコンテキスト

### コンテキスト処理を備えた Strands エージェント

セッション管理とコンテキスト処理を実証する実装を見てみましょう：

In [ ]:
%%writefile strands_claude_context.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool

# 計算機ツールをインポート
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# カスタムツールの作成
@tool
def weather():
    """ 天気を取得する """ # ダミー実装
    return "sunny"

@tool
def get_time():
    """ 現在時刻を取得する """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""
    あなたは役立つアシスタントです。簡単な計算ができ、
    天気を伝え、現在の時刻を提供することができます。
    常にユーザーの名前を確認することから始めてください
    """
)

def get_user_name(user_id):
    users = {
        "1": "Maira",
        "2": "Mani",
        "3": "Mark",
        "4": "Ishan",
        "5": "Dhawal"
    }
    return users[user_id]
    
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    """
    AgentCore Runtime のエントリーポイントで、コンテキスト処理とセッション管理を実演します。
    
    Args:
        payload: ユーザーデータとリクエスト情報を含む入力ペイロード
        context: セッションと実行情報を含むランタイム コンテキスト オブジェクト
    
    Returns:
        str: コンテキスト情報を組み込んだエージェントの応答
    """
    user_input = payload.get("prompt")
    user_id = payload.get("user_id")
    user_name = get_user_name(user_id)
    
    # ランタイムコンテキスト情報へのアクセス
    print("=== Runtime Context Information ===")
    print("User id:", user_id)
    print("User Name:", user_name)
    print("User input:", user_input)
    print("Runtime Session ID:", context.session_id)
    print("Context Object Type:", type(context))
    print("=== End Context Information ===")
    
    # パーソナライズされたプロンプトを作成し、コンテキスト情報を含める
    prompt = f"""私の名前は {user_name} です。こちらが私のリクエストです: {user_input}
    
    追加の文脈: これはセッション {context.session_id} です。
    私の名前を確認して、サポートを提供してください。"""
    
    response = agent(prompt)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

# AgentCore ランタイムにおけるセッション管理の理解

上記のコードは、AgentCore ランタイムがどのようにセッションを管理し、エージェントにコンテキストを提供するかについていくつかの重要な概念を示しています：

### コンテキストオブジェクトの構造

エントリーポイント関数の `context` パラメータは、ランタイム情報へのアクセスを提供します：

```python
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    # セッション情報へのアクセス
    session_id = context.session_id
    # エージェントロジックでコンテキスト情報を使用
```

### セッション継続性の利点

単一のセッション内で、AgentCore ランタイムは以下を提供します：

1. **永続的な環境**：変数と状態が呼び出し間で持続します
2. **コンテキストの保持**：エージェントは以前のやり取りを参照できます
3. **リソースの再利用**：初期化されたモデルとツールはロードされたままです
4. **パフォーマンスの向上**：後続の呼び出しのためのコールドスタート時間の短縮

### セッション分離の保証

AgentCore ランタイムは、セッション間の完全な分離を保証します：

- **セキュリティ**：各セッションは独自のマイクロ VM で分離されたリソースで実行されます
- **プライバシー**：異なるユーザーセッション間でのデータ漏洩はありません
- **信頼性**：あるセッションの問題が他のセッションに影響しません
- **クリーンアップ**：セッション終了後の完全なメモリサニタイズ

### ペイロードの柔軟性

`payload` パラメータは柔軟なデータ受け渡しを可能にします：

```python
# ペイロード構造の例
payload = {
    "prompt": "ユーザーの質問",
    "user_id": "1",
    "preferences": {...},
    "context_data": {...}
}
```

これにより、ランタイムによって提供されるセッションコンテキストを維持しながら、クライアントとエージェント間の豊かで構造化された通信が可能になります。

### ランタイムロールの作成

始める前に、AgentCore ランタイム用の IAM ロールを作成しましょう。これは、あなたのために事前に開発されたユーティリティ関数を使用して行います。

In [ ]:
import sys
import os

# 現在のノートブックのディレクトリを取得する
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# utils.py の場所に移動する
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)

from utils import create_agentcore_role

agent_name="strands_claude_context"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

### AgentCore Runtime デプロイメントの構成

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、および要件ファイルを含む AgentCore Runtime デプロイメントを構成します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを構成します。

構成ステップでは、アプリケーションコードに基づいて Docker ファイルが生成されます

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_context.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)

### AgentCore Runtime へのコンテキスト対応エージェントの起動

Docker ファイルができたので、コンテキスト対応エージェントを AgentCore Runtime に起動しましょう。これにより Amazon ECR リポジトリと AgentCore Runtime が作成されます。

私たちのエージェントは、AgentCore Runtime がセッションを管理し、エージェントにコンテキスト情報を提供する方法を実演します。

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore ランタイムのステータス確認
AgentCore ランタイムをデプロイしたので、そのデプロイ状況を確認しましょう

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

# セッション管理とコンテキスト処理のデモンストレーション

ここでは、さまざまなシナリオをテストすることで AgentCore Runtime の主要なセッション管理機能をデモンストレーションします：

### シナリオ 1: セッションの継続性
複数の呼び出しで同じセッション ID を使用して、コンテキストがどのように維持されるかを示します。

### シナリオ 2: セッションの分離
異なるセッション ID を使用して、セッション間の完全な分離を実証します。

### シナリオ 3: コンテキスト情報へのアクセス
エージェントがランタイムコンテキスト情報にアクセスする方法を示します。

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [ ]:
import uuid
import json
from IPython.display import Markdown, display

# セッションの継続性を実証するためのセッション ID を作成する
session_id = uuid.uuid4()
print(f"📋 Starting Session 1: {session_id}")
print(f"👤 User: Maira (ID: 1)")
print(f"❓ First question about weather\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "外の天気はどうですか?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = json.loads(invoke_response['response'][0])
display(Markdown(response_data))

In [ ]:
# 同じセッション ID を継続して、セッションの継続性を示す
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Follow-up question about math\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "2+5の答えは?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = json.loads(invoke_response['response'][0])
display(Markdown(response_data))

In [ ]:
# 同じセッション ID で続ける - エージェントが前回の計算を覚えていることに注目してください
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Building on previous answer - demonstrates context continuity\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "それに 34 を足すとどうなりますか?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = json.loads(invoke_response['response'][0])
display(Markdown(response_data))

In [ ]:
# 新しいセッション - セッション分離のデモンストレーション
# コンテキストが失われることを示すために完全に新しいセッション ID を作成する
new_session_id = uuid.uuid4()
print(f"🆕 Starting NEW Session 2: {new_session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Attempting to reference previous calculation - should fail due to session isolation\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "そしてそれにさらに 10 を足すとどうなりますか?",
    "user_id": "1"
}, session_id=str(new_session_id))

response_data = json.loads(invoke_response['response'][0])
display(Markdown(response_data))

In [ ]:
# 新しいセッションとユーザー - 完全な分離を実証
different_user_session = uuid.uuid4()
print(f"🆕 Starting Session 3: {different_user_session}")
print(f"👤 Different user: Mani (ID: 2)")
print(f"❓ Same question as first user - demonstrates user isolation\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "天気はどうですか？",
    "user_id": "2"
}, session_id=str(different_user_session))

response_data = json.loads(invoke_response['response'][0])
display(Markdown(response_data))

## セッション管理結果の理解

上記のデモンストレーションは、AgentCore Runtimeのセッション管理の重要な側面をいくつか示しています：

### 1. セッションの継続性（セッション 1）
- **最初の呼び出し**：エージェントは天気の質問に応答し、ユーザー名を認識します
- **2回目の呼び出し**：エージェントは計算を実行します（2×5=10）
- **3回目の呼び出し**：エージェントは前回の結果を参照します（「それに34を足すと」= 44）

**重要な学び**：エージェントは同じセッション内の複数の呼び出しにわたってコンテキストを維持し、前回のやり取りでの計算結果を記憶していました。

### 2. セッションの分離（セッション 2）
- **新しいセッションID**：完全に新しいセッションを作成
- **同じユーザー**：同じユーザーIDを使用したが、異なるセッション
- **コンテキストの喪失**：エージェントは前回の計算を参照できません

**重要な学び**：同じユーザーであっても、新しいセッションは前のコンテキストにアクセスできない完全に分離された環境を作成します。

### 3. ユーザーとセッションの分離（セッション 3）
- **異なるユーザー**：MairaではなくMani
- **新しいセッション**：前のセッションからの完全な分離
- **新しいコンテキスト**：エージェントはクリーンな状態から開始します

**重要な学び**：各セッションは完全な分離を提供し、異なるユーザーやインタラクション間のプライバシーとセキュリティを確保します。

### 4. コンテキストオブジェクトの使用
すべての呼び出しを通じて、エージェントは：
- `context.session_id` を通じてランタイムコンテキストにアクセス
- カスタムペイロードデータ（`user_id`、`prompt`）を処理
- ロギングとデバッグ情報を維持

**重要な学び**：コンテキストオブジェクトは、エージェントが機能強化とデバッグに使用できる貴重なランタイム情報を提供します。

### 実証されたセッション管理のベストプラクティス

1. 会話の継続性のために**一貫したセッションID**を使用する
2. 異なるユーザーや会話には**一意のセッションID**を生成する
3. エージェントの動作を強化するために**コンテキスト情報を活用**する
4. **セッション境界を考慮した設計**をする - セッション間の永続性を前提としない
5. セッションが変更または期限切れになった場合の**コンテキスト喪失を適切に処理**する

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
import boto3

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# おめでとうございます！

Amazon Bedrock AgentCore Runtime を使用したセッション管理とコンテキスト処理の実装とテストに成功しました！

## 学んだこと：

### セッション管理の基礎
* **セッションの継続性**：同じセッション ID が複数の呼び出しにわたってコンテキストを維持する方法
* **セッションの分離**：異なるセッション ID が完全に分離された環境を作成する方法
* **コンテキストの保存**：エージェントが状態を維持し、以前のやり取りを参照する方法
* **セキュリティ境界**：AgentCore Runtime がユーザー間の完全な分離を確保する方法

### ランタイムコンテキスト処理
* **コンテキストオブジェクトへのアクセス**：`context` パラメータを通じてランタイム情報にアクセスする方法
* **セッション情報**：エージェントロジックでセッション ID を取得して使用する方法
* **ペイロード処理**：カスタムペイロードを通じて渡される構造化データを処理する方法
* **ランタイムメタデータ**：エージェントが実行環境情報にアクセスする方法

### AgentCore Runtime アーキテクチャ
* **MicroVM 分離**：各セッションは独自の分離された microVM で実行される
* **リソース管理**：セッションごとに専用の CPU、メモリ、ファイルシステム
* **セキュリティモデル**：セッション終了後の完全なメモリサニタイズ
* **ライフサイクル管理**：セッション状態（アクティブ、アイドル、終了）とタイムアウト

### ベストプラクティスの実装
* **セッション ID の生成**：異なる会話のための一意の識別子の作成
* **コンテキストの活用**：エージェントの動作を強化するためのランタイムコンテキストの活用
* **状態管理**：一時的な状態と永続的な状態の理解
* **エラー処理**：コンテキスト喪失とセッション境界の適切な処理